In [1]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader
import random
from dataloader import SkinCancerPredictionDatasetLoader

from model import NeuralNetwork

device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)

torch.cuda.empty_cache()

/tmp/ipykernel_238271/145120561.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/adria/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Train and test loops

In [3]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        current_y, current_pred = [], []
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)

            current_y.append(y)
            current_pred.append(pred)

            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    return current_y, current_pred, correct

def train_loop(dataloader, model, loss_fn, optimizer, batch_size):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def get_sensitivity(y_reals, y_preds): # Habilidad para decirte si estas enfermo
    true_positive = 0
    false_negative = 0
    for i in range(len(y_preds)):
        if y_preds[i] == 1 and y_reals[i] == 1:
            true_positive += 1
        if y_preds[i] == 0 and y_reals[i] == 1:
            false_negative += 1
    return true_positive / ( true_positive + false_negative )
 
def get_specificity(y_reals, y_preds): # Habilidad para decirte que NO estas enfermo
    true_negative = 0
    false_positive = 0
    for i in range(len(y_preds)):
        if y_preds[i] == 0 and y_reals[i] == 0:
            true_negative += 1
        if y_preds[i] == 1 and y_reals[i] == 0:
            false_positive += 1
    return true_negative / ( true_negative + false_positive )

In [4]:
def main(loop_num):
    print(f'Running {loop_num} loop....')
    batch_size = 50
    learning_rate = 0.0001
    epochs = 15


    train_dataset = SkinCancerPredictionDatasetLoader('./train')
    test_dataset = SkinCancerPredictionDatasetLoader('./test')

    train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size, shuffle=True)

    model = NeuralNetwork().to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}\n ---------------------------")
        train_loop(train_loader, model, loss_fn, optimizer, batch_size)
        test_loop(test_loader, model, loss_fn)
        
    torch.save(model.state_dict(), f'models/model{random.randint(0,10000)}.pth')
    

In [5]:
for i in range(0,10):
  main(1)

Running 1 loop....
Epoch 1
 ---------------------------
loss: 3.390588  [   50/10548]


KeyboardInterrupt: 